In [68]:
from os import listdir
#from os.path import isfile, join
from pathlib import Path         # python 3 way
import re
import sys
import string
import csv
import pickle
import matplotlib
import math

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

import pandas as pd
import numpy as np

import Settings as S                    # pathnames
from CM0645db import Db

In [62]:
class Data_Clean:
    df = None

    def __init__(self, db):
        '''Initialize class and get pandas dataframe'''
        self.DB = db
        self.df = DB.getDF('projects')
        self.df.describe()
        
    
        
      #generic normalizer/scaler  assumes all cols numeric
  # #   
    def NormDataFrames(self, df):
        #from sklearn.preprocessing import StandardScaler
        # cols_to_norm = df100a.columns[1:]
        # df100a[cols_to_norm] = MinMaxScaler().fit_transform(df100a[cols_to_norm])
        scaler = MinMaxScaler()
        scaled_values = scaler.fit_transform(df) 
        df.loc[:,:] = scaled_values
        # Remove Outliers
        ndf = df[(np.abs(stats.zscore( df)) < 4).all(axis=1)]
        return ndf
    
    def DataFramesRemoveBlank(self, df):
        #https://stackoverflow.com/questions/21164910/delete-column-in-pandas-if-it-is-all-zeros
        ndf = df.loc[:, (df != 0).any(axis=0)]
        # Drop a row by condition
        ndf2 = ndf[df.AWL_count != 0] #nothing read if AWL is zero
        #remove other nearly empty cols or with minimal data
        #ndf2 = ndf.drop(['tag_LS', 'tag_SYM', 'tag_NNPS', 'tag_UH', 'tag_D'], axis=1)
        return ndf2
    

In [10]:
bd = Path('/home/jeremy/Projects_Active/')
print(bd)

/home/jeremy/Projects_Active


In [13]:
dbfile = bd /'CM0645.sqlite'

In [14]:
DB = Db(dbfile)

SQLite Version OK 2.6.0


In [41]:
DC = Data_Clean(DB)

In [43]:
df = DC.df

In [63]:
DC.df

,uid,first_name,last_name,cohort,filename,Report_Mark,Report_Max,Report_Percent,AWL_count,CSAWL_count,...,NOUN,NUM,PART,PRON,PROPN,PUNCT,SPACE,SYM,VERB,X
0,8025209,PAUL JAMES,CUTHBERT,15-16,Paul_Cuthbert-Paul_Cuthbert_-_Project_Report.txt,39.00,60,65.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,9003473,ANDREW PETER,JONES,17-18,Jordan_Jones-Unf_Dissertation_-_Jordan_Jones_-...,36.60,60,61.000000,1168.0,251.0,...,22.867988,1.417124,2.083509,1.231548,5.078026,8.814846,2.134121,0.194011,16.887389,0.033741
2,10004990,JOHN STEVEN BRAVO,MARTINEZ,15-16,John_Martinez-Individual_Project_with_Appendic...,44.00,60,73.333333,2708.0,297.0,...,22.501050,3.513999,1.841030,0.958711,11.706963,9.560368,5.515450,0.332302,14.029258,0.072572
3,10006962,ROSS ALISTAIR,BROWN,17-18,Ross_Brown-Final_Individual_project_report.txt,36.50,60,60.833333,1873.0,467.0,...,24.380021,1.037872,3.108627,1.357218,2.095704,8.327928,2.115663,0.309366,19.205628,0.493987
4,10011846,RICHARD,DANBY,15-16,Richard_Danby-full_Report_e_hand_in_only.txt,36.60,60,61.000000,2891.0,1164.0,...,22.050073,1.552975,1.857765,1.128447,7.412917,6.502177,1.843251,0.141509,17.674165,0.068940
5,10013249,DAVE,APPLEBY,16-17,Dave_Appleby-Dave_Appleby_-_Dissertation.txt,42.50,60,70.833333,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,10020946,MICHAEL,MCVEIGH,15-16,Michael_McVeigh-individual-project_final.txt,42.00,90,46.666667,2326.0,511.0,...,23.283363,1.043086,1.944648,1.132194,3.097809,11.363875,2.453087,0.183457,17.234511,0.015725
7,10021240,STEPHEN,DALZELL,15-16,Stephen_Dalzell-Stephen_Dalzell_W10021240_Diss...,41.00,60,68.333333,3000.0,1286.0,...,19.330672,5.759902,2.575990,0.589500,10.921093,8.492478,4.249309,0.085969,17.890697,0.018422
8,11002506,JOSEPH,CRIPPS,15-16,Matt_Cripps-A_Forensic_Comparsion_of_Windows_7...,31.00,60,51.666667,1472.0,309.0,...,20.434872,3.844319,1.308704,1.581351,7.102447,11.764706,5.330243,0.047713,14.988753,0.000000
9,11003225,NICOLE,MATTHEWS,15-16,Nicole_Matthews-Final_Year_Project_-_CM0645_-_...,74.00,90,82.222222,2921.0,537.0,...,22.478007,2.379588,2.258249,1.543699,7.843203,12.177694,4.061478,0.259530,14.911187,0.050558


In [71]:
ndf = df.iloc[:, df.columns.get_loc('Report_Percent'):]
ndf2 = ndf[df.AWL_count != 0]
grade = np.floor(ndf2['Report_Percent']/10)
ndf

,Report_Percent,AWL_count,CSAWL_count,LFP_k1,LFP_k2,LFP_k3,LFP_k4,LFP_kn,LFP_kx,n_sents,...,NOUN,NUM,PART,PRON,PROPN,PUNCT,SPACE,SYM,VERB,X
0,65.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,61.000000,1168.0,251.0,0.258339,0.191116,0.123894,0.065861,0.269571,0.091219,341,...,22.867988,1.417124,2.083509,1.231548,5.078026,8.814846,2.134121,0.194011,16.887389,0.033741
2,73.333333,2708.0,297.0,0.220946,0.266133,0.122336,0.068536,0.238854,0.083195,1223,...,22.501050,3.513999,1.841030,0.958711,11.706963,9.560368,5.515450,0.332302,14.029258,0.072572
3,60.833333,1873.0,467.0,0.267012,0.225062,0.139868,0.050253,0.215896,0.101909,659,...,24.380021,1.037872,3.108627,1.357218,2.095704,8.327928,2.115663,0.309366,19.205628,0.493987
4,61.000000,2891.0,1164.0,0.230215,0.187577,0.100000,0.069939,0.324847,0.087423,768,...,22.050073,1.552975,1.857765,1.128447,7.412917,6.502177,1.843251,0.141509,17.674165,0.068940
5,70.833333,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,46.666667,2326.0,511.0,0.242872,0.251085,0.101021,0.051390,0.273495,0.080136,613,...,23.283363,1.043086,1.944648,1.132194,3.097809,11.363875,2.453087,0.183457,17.234511,0.015725
7,68.333333,3000.0,1286.0,0.205770,0.265380,0.088214,0.072741,0.301751,0.066145,1837,...,19.330672,5.759902,2.575990,0.589500,10.921093,8.492478,4.249309,0.085969,17.890697,0.018422
8,51.666667,1472.0,309.0,0.220401,0.219412,0.131534,0.040407,0.306442,0.081803,442,...,20.434872,3.844319,1.308704,1.581351,7.102447,11.764706,5.330243,0.047713,14.988753,0.000000
9,82.222222,2921.0,537.0,0.282135,0.176954,0.093998,0.043672,0.287939,0.115303,1231,...,22.478007,2.379588,2.258249,1.543699,7.843203,12.177694,4.061478,0.259530,14.911187,0.050558


In [77]:
ndf = df.iloc[:, df.columns.get_loc('Report_Percent'):]
ndf4 = DC.DataFramesRemoveBlank(ndf)
#ndf5 = ndf4.iloc[:, ndf4.columns.get_loc('AWL_count'):]
ndf6 = DC.NormDataFrames( ndf4.iloc[:, ndf4.columns.get_loc('AWL_count'):])
ndf6.insert(loc=0, column='Grade', value =ndf4.Report_Percent)
#ndf7.Report_Percent = np.floor(ndf7.Report_Percent/10)
ndf6.Grade = np.floor(ndf6.Grade/10)
ndf6.to_csv("testout.csv")

/home/jeremy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
